In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow.compat.v1 as tf
import pprint as pp
ds,info = tfds.load('mnist', split='train', shuffle_files=True,with_info=True)
train_images, train_labels = tfds.as_numpy(tfds.load(
    'mnist',
    split='train', 
    batch_size=-1, 
    as_supervised=True,
))

train_images = (train_images / 255)
train_images = train_images.reshape((-1, 784))

depth = 10
train_labels = tf.one_hot(train_labels, depth).numpy()

train_images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [2]:
num_epochs = 5
learning_rate = 0.001
batch_size = 50

In [3]:
def get_batch(X,Y, batch_size):
    index = np.random.randint(0, len(X), batch_size)
    return X[index, :], Y[index]
num_batches = int(len(train_images) // batch_size * num_epochs)

In [4]:
class Model(object):
    def __init__(self, input_size, label_size, activation, num_layers=6,
                 hidden_size=10):
        self._input_size = input_size
        self._label_size = label_size
        self._activation = activation
        # num layers does not include the input layer
        self._num_layers = num_layers
        self._hidden_size = hidden_size
        self._model_def()

    def _model_def(self):
        
        self.input_images = tf.placeholder(tf.float32, shape=[None, self._input_size])
        self.labels = tf.placeholder(tf.float32, shape=[None, self._label_size])
        
        input = self.input_images
        for i in range(self._num_layers - 1):
            input = tf.layers.dense(input, self._hidden_size, activation=self._activation,name='layer{}'.format(i+1))
            
            
        logits = tf.layers.dense(input, 10, name='layer{}'.format(self._num_layers))
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=self.labels))

        
        tf.summary.scalar('loss', self.loss)
        
        self._log_gradients(self._num_layers)
        
        self.optimizer = tf.train.AdamOptimizer().minimize(self.loss)
        
        self.accuracy = self._compute_accuracy(logits, self.labels)
        tf.summary.scalar('acc', self.accuracy)
        
        self.merged = tf.summary.merge_all()
        self.init_op = tf.global_variables_initializer()

    def _compute_accuracy(self, logits, labels):
        prediction = tf.argmax(logits, 1)
        equality = tf.equal(prediction, tf.argmax(labels, 1))
        accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
        return accuracy

    def _log_gradients(self, num_layers):
        gr = tf.get_default_graph()
        
        for i in range(num_layers):
            weight = gr.get_tensor_by_name('layer{}/kernel:0'.format(i + 1))
            grad = tf.gradients(self.loss, weight)[0]
            mean = tf.reduce_mean(tf.abs(grad))
            tf.summary.scalar('mean_{}'.format(i + 1), mean)
            tf.summary.histogram('histogram_{}'.format(i + 1), grad)
            tf.summary.histogram('hist_weights_{}'.format(i + 1), grad)
base_path = "./Tensorboard/"           
def run_training(model, X,Y, sub_folder, iterations=3333, batch_size=30):
    with tf.Session() as sess:
        sess.run(model.init_op)
        
        train_writer = tf.summary.FileWriter(base_path + sub_folder,sess.graph)
        
        for i in range(iterations):
            image_batch, label_batch = get_batch(X,Y,batch_size)
            l, _, acc = sess.run([model.loss, model.optimizer, model.accuracy],
                                 feed_dict={model.input_images: image_batch, model.labels: label_batch})
            if i % 200 == 0:
                summary = sess.run(model.merged, feed_dict={model.input_images: image_batch,
                                                            model.labels: label_batch})
                train_writer.add_summary(summary, i)
                print("Iteration {} of {}, loss: {:.3f}, train accuracy: "
                      "{:.2f}%".format(i, iterations, l, acc * 100))

In [ ]:
tf.compat.v1.disable_eager_execution()
scenarios = ["sigmoid", "relu", "leaky_relu"]
act_funcs = [tf.sigmoid, tf.nn.relu, tf.nn.leaky_relu]

X = train_images
Y = train_labels

for i in range(len(scenarios)):
    tf.reset_default_graph()
    print("Running scenario: {}".format(scenarios[i]))
    model = Model(784, 10, act_funcs[i], 20, 10)
    run_training(model, X,Y, scenarios[i])

W0627 19:58:52.733618 4451820992 deprecation.py:323] From <ipython-input-4-d1fbd16f6ecc>:19: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.
W0627 19:58:52.742202 4451820992 deprecation.py:323] From /Users/shaunyang/anaconda3/envs/tensorflow_ENV/lib/python3.7/site-packages/tensorflow/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W0627 19:58:52.763722 4451820992 deprecation.py:506] From /Users/shaunyang/anaconda3/envs/tensorflow_ENV/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:1666: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras p

Running scenario: sigmoid
Iteration 0 of 3333, loss: 2.321, train accuracy: 6.67%
Iteration 200 of 3333, loss: 2.307, train accuracy: 6.67%
Iteration 400 of 3333, loss: 2.289, train accuracy: 16.67%
Iteration 600 of 3333, loss: 2.300, train accuracy: 10.00%
Iteration 800 of 3333, loss: 2.283, train accuracy: 16.67%
Iteration 1000 of 3333, loss: 2.291, train accuracy: 16.67%
Iteration 1200 of 3333, loss: 2.323, train accuracy: 3.33%
Iteration 1400 of 3333, loss: 2.297, train accuracy: 13.33%
Iteration 1600 of 3333, loss: 2.331, train accuracy: 13.33%
Iteration 1800 of 3333, loss: 2.319, train accuracy: 3.33%
Iteration 2000 of 3333, loss: 2.295, train accuracy: 16.67%
Iteration 2200 of 3333, loss: 2.313, train accuracy: 10.00%
Iteration 2400 of 3333, loss: 2.301, train accuracy: 16.67%
Iteration 2600 of 3333, loss: 2.309, train accuracy: 6.67%
Iteration 2800 of 3333, loss: 2.304, train accuracy: 6.67%
Iteration 3000 of 3333, loss: 2.302, train accuracy: 10.00%
Iteration 3200 of 3333, los

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="./Tensorboard"